# Modeling a reaction by mass action cascades

This notebook is part of the publication "EnzymeML at Work" from Lauterbach et al. 2022 and compares the fitting of a micro-kinetic model (specified in an EnzymeML document) to experimental data (specified in the same EnzymeML document). 

Generation of the EnzymeML document and individual fitting of the data with either PySCeS or COPASI have been dealt with in separate notebooks.

-----
## Comparison of modelling with PySCeS and COPASI

Now that the EnzymeMLDocument has been adapted to the micro-kinetic model, it can be modeled and optimized using PySCeS and COPASI. Since both modeling package interfaces are an integral part of PyEnzyme, called Thin Layer, a simple call to the corresponding Thin Layer object is necessary.

But before optimization, it might be necessary to define initial values. Since manipulating the KineticParameter initial_values attributes inside the script that generates the EnzymeMLDocument can get quite tedious, PyEnzyme offers an external data structure from within initial values can be applied. This way, the EnzymeML document is only modifed at optimization and remains untouched until then.

The initialization file is in the YAML format and contains all reactions and their parameters. 

In [1]:
# Load the EnzymeML Document from file
from pyenzyme import EnzymeMLDocument

enzmldoc = EnzymeMLDocument.fromFile("Model_4.omex")

### Modelling with the PySCeS thin layer

Thin Layers require to follow a given metaclass and thus the syntax of every modeling layer follows the Initialization > ```optimize```-method > ```write```-method procedure. 

In [2]:
from pyenzyme.thinlayers import ThinLayerPysces

In [3]:
# Initialize the layer
tl_pysces = ThinLayerPysces(
    "Model_4.omex", 
    init_file="EnzymeML_Lagerman_init_values_.yaml",
    model_dir="pySCeS"
)

Check SBML support is at action level 2
SBML file is L3V2



*********ERRORS***********


*********ERRORS***********


Possible errors detected in SBML conversion, Model may be incomplete. Please check the error log file "EnzymeML_Lagerman.xml-sbml_conversion_errors.txt" for details.


*******************************************************************
Issues encountered in SBML translation (model processed anyway)
SBML source: pySCeS/EnzymeML_Lagerman.xml
*******************************************************************

Parameter units ignored for parameters:
['v_r', 'K_si', 'K_n'] 

Parameter units ignored for (local) parameters:
['K_s', 'k_2', 'k_6', 'k_3', 'K_pg', 'k_5', 'k_4', 'k_4b', 'K_p', 'k_d'] 

*******************************************************************

Info: single compartment model: locating "r0" in default compartment
Info: single compartment model: locating "r1" in default compartment
Info: single compartment model: locating "r2" in default compartment
Info: 

In [4]:
# Run optimization
tl_pysces.model.mode_integrator='CVODE'
tl_opt = tl_pysces.optimize(method="least_squares")

# Write to new EnzymeMLDocument and save
pysces_doc = tl_pysces.write()
pysces_doc.toFile(".", name="EnzymeML_Lagerman_M4_PySCeS_Modeled")


Archive was written to ./EnzymeML_Lagerman_M4_PySCeS_Modeled.omex



### Modelling with the COPASI thin layer

In the same manner the COPASI Thin Layer can be used to model the given data. 

The COPASI optimization is set up to use the same initial values and the same fitting algorithm that was used with PySCeS, to allow an easy comparison.

In [5]:
from basico import set_current_model, set_task_settings, T, PE
from pyenzyme.thinlayers import ThinLayerCopasi

In [6]:
# Initialize COPASI Thin Layer
tl_copasi = ThinLayerCopasi(
    "Model_4.omex", "COPASI",
    init_file="EnzymeML_Lagerman_init_values_.yaml"
)

set_current_model(tl_copasi.dm)
set_task_settings(T.PARAMETER_ESTIMATION, 
                  {
                      'scheduled': True,
                      'problem': {'Randomize Start Values': False},
                      'method': {'name': PE.LEVENBERG_MARQUARDT}
                  })

tl_copasi.optimize()

In [7]:
copasi_doc = tl_copasi.write()
copasi_doc.toFile(".", name="EnzymeML_Lagerman_M4_COPASI_Modeled")

DEBUG:pyenzyme.thinlayers.TL_Copasi:OBJ: 504.10317547262457
DEBUG:pyenzyme.thinlayers.TL_Copasi:RMS: 1.2551184897657877
DEBUG:pyenzyme:KineticParameter 'K_si' - value was set from '6.346729249' to '6.3097521949280955'
DEBUG:pyenzyme:KineticParameter 'K_n' - value was set from '11.00542741' to '10.993443343422388'
DEBUG:pyenzyme:KineticParameter 'K_s' - value was set from '5.676255758' to '5.66994298385956'
DEBUG:pyenzyme:KineticParameter 'k_2' - value was set from '652085.5139' to '651689.4527365268'
DEBUG:pyenzyme:KineticParameter 'k_6' - value was set from '393169.3931' to '392326.3186840926'
DEBUG:pyenzyme:KineticParameter 'k_3' - value was set from '8.93817854' to '8.745658858174288'
DEBUG:pyenzyme:KineticParameter 'K_pg' - value was set from '47.34446037' to '48.70176101767321'
DEBUG:pyenzyme:KineticParameter 'k_5' - value was set from '672295.823' to '669178.187376541'
DEBUG:pyenzyme:KineticParameter 'k_4' - value was set from '1870570.524' to '1866816.714995926'
DEBUG:pyenzyme:K


Archive was written to ./EnzymeML_Lagerman_M4_COPASI_Modeled.omex



### Comparison of results

Both results can now be compared by individually exporting the estimated parameters using the ```exportKineticParameters```-method found in the ```EnzymeMLDocument``` instance that returns a Pandas ```DataFrame``` object. Finally, for the sake of comparison, both result are merged into a single ```DataFrame```.

In [8]:
params = pysces_doc.exportKineticParameters(exclude_constant=True)
params.rename({"value": "PySCeS"}, axis="columns", inplace=True)
params["COPASI"] = copasi_doc.exportKineticParameters(exclude_constant=True).value

params[["name", "PySCeS", "COPASI", "unit"]]

,name,PySCeS,COPASI,unit
reaction,,,,
r1,K_s,5.129503e+00,5.669943e+00,mmole / l
r2,k_2,5.694527e+05,6.516895e+05,1 / min
r5,k_6,2.495553e+05,3.923263e+05,1 / min
r6,k_3,1.589958e+01,8.745659e+00,1 / min
r7,K_pg,1.299523e+02,4.870176e+01,mmole / l
r9,k_5,8.846055e+05,6.691782e+05,1 / min
r10,k_4,1.577461e+06,1.866817e+06,1 / min
r10,k_4b,3.680255e+04,4.253530e+04,1 / min
r11,K_p,1.295747e+00,9.339814e-01,mmole / l


-------